# 深度前馈网络

**深度前馈网络(deep feedforward network)**,也叫作 **前馈神经网络(feedforward neural network)** 或者 **多层感知机(multilayer perceptron,MLP)**。

## 最简单的模型

在介绍深度前馈网络之前，我们先介绍神经网络的基本单元——**神经元模型**。

### 神经元模型

我们采用如下关于**神经网络**的定义：

    神经网络是由具有适应性的简单单元组成的广泛并行互联的网络，它的组织能够模拟神经系统对真实世界物体所作出的交互反应。

neural networks 中最基本的成分是 神经元(neuron)模型，即上述定义中的“简单单元”。模型借鉴如下生物神经网络的特性：

    生物神经网络中，每个神经元与其他神经元相连，当它“兴奋”时，就会向相连的神经元发送化学物质，从而改变这些神经元内的电位；如果某神经元的电位超过了一个“阈值”(threshold),那么它就会被激活，即“兴奋”起来，向其他神经元发送化学物质。

上述特性被 McCulloch and Pitts 抽象为如下的简单模型，也就是一直沿用的**“M-P 神经元模型”**：

    在这个模型中，神经元接收到来自 n 个其他神经元传递过来的输入信号，这些输入信号通过带权重的连接进行传递，神经元接收到的总输入值与神经元的阈值进行比较，然后通过“激活”函数处理以产生神经元的输出。

<img src='figure/mp-neuron.png' width='400' height='300' align='left'>

神经元模型的输出数学表达如下：

$$y = f\left(\sum_{i=1}^n w_i x_i - \theta\right)$$

其中：

* $x_i$ 是第 i 个神经元的输入；
* $w_i$ 是第 i 个神经元到该神经元的连接权重；
* $\theta$ 是该神经元的阈值；
* f 是负责输出的“激活函数”。

**激活函数 f**

理想的激活函数是下图左侧的阶跃函数，将输入值映射为输出值 0 或 1,对应着神经元的“抑制”与“兴奋”。但是阶跃函数具有不连续、不光滑等不太好的性质，所以实际上常用 Sigmoid 函数作为激活函数，见下图右侧。Sigmoid 函数可以将较大范围内的输入挤压到(0,1)输出值范围内。

<img src='figure/act-fun.png' width='400' height='300' align='left'>

### 单层感知机

感知机（Perceptron）由两层神经元组成，如下图所示：

    输入层接收外界输入信号传递给输出层；输出层是M-P 神经元，也称为“阈值逻辑单元”。

<img src='figure/perceptron.png' width='400' height='300' align='left'>

**案例：与、或、非运算的实现**

感知机可以实现逻辑与、或、非运算。假定 f 是阶跃函数。对应的三种运算为

|逻辑运算|参数$(w_1, w_2,\theta)$|$\quad$对应的数学模型 $y = f\left(\sum_{i=1}^n w_i x_i - \theta\right)$|$\quad$实际运算$\quad$|
| :---: |:---:|:---:|:---:|
|与：$x_1 \land x_2$|(1,1,2)|$y=f(x_1+x_2-2)$|在 $x_1=x_2=1$时，y=1|
|或：$x_1 \lor x_2$|(1,1,0.5)|$y=f(x_1+x_2-0.5)$|在 $x_1=1$ 或 $x_2=1$ 时，y=1|
|非：$\urcorner x_1\;\quad$|(-0.6,0,-0.5)|$y=f(-0.6x_1-0.5)$|在 $x_1=0$时，y=1|

更一般地，给定训练数据集，权重 $w_i \;(i=1,2,\cdots,n)$ 以及阈值 $\theta$ 可以通过学习得到。阈值 $\theta$ 可看做一个固定输入为 $-1.0$ 的“哑节点(dummy node)”所对应的连接权重 $w_{n+1}$，权重和阈值在这样的设定下可以统一为权重的学习。

**学习规则**

对训练样例 $(\mathbf{x},y)$，假设当前感知机的输出为 $\hat{y}$，则感知机权重调整方式为：

$$w_i\gets w_i + \eta(y-\hat{y})x_i$$

其中 $\eta$ 称为学习率。

**感知机的局限性**

感知机只有输出层神经元进行激活函数处理，即**只拥有一层功能神经元**，学习能力非常有限。

事实上，上面案例中的与或非问题都是线性可分问题。可以证明：若两类模式是线性可分的，即存在一个线性超平面能将它们分开，如下图的 (a)-(c) 所示，则感知机的学习过程一定会收敛而求得适当的权向量 $\mathbf{w} = (w_1,w_2, \cdots,w_{n+1})$；如果是非线性可分的，则感知机学习过程将会发生振荡， $\mathbf{w}$ 难以稳定下来。

例如感知机甚至不能解决下图所示的 (d) 所示的**异或**这样简单的非线性可分问题。

<img src='figure/linear.jpg' width='400' height='300' align='left'>

要解决异或问题，需要引出我们今天的主题——**多层感知机，也就是深度前馈网络。**

## 多层感知机(MLP)定义

* **前向**：模型被称为**前向(feedforward)**的，是因为模型的输出和模型本身之间没有**反馈**连接。如果被扩展成反馈连接时，对应的被称为**循环神经网络**；


* **网络**：前馈神经网络之所以被称作**网络**，是因为它通常用许多不同函数复合在一起所形成的复合映射：
$$f(\mathbf{x}) = f^{(n)} \circ \cdots \circ f^{(2)}\circ f^{(1)}(\mathbf{x})$$
从输入层开始，具有激活函数功能的 $f^{(k)}$ 称为第 k 层，其实质含义是**以第 k-1 层的输出作为输入，k-1 层到第 k 层之间的神经元连接权重所组成的权重矩阵，阈值向量，激活函数所一起构成的**。该模型对应的拓扑结构是一个有向无环图，如下图所示。


* **隐藏层**：前馈神经网络的最后一层被称为**输出层**，介于输入层与输出层之间训练数据没有给出真实输出的层被称为**隐藏层**；


* **深度**：我们定义一个前馈神经网络的**深度**为该网络所包含的**隐层层数**。正是因为“深度”这个术语，才出现了“深度学习”这个名字，神经网络要学习的就是不同层之间的连接权重以及阈值。


**注**：“前馈”并不意味着网络中信号不能向后传播，不然BP算法也就不能用了，而是指网络拓扑结构上不存在环或者回路。

<img src='figure/yinceng.png' width='400' height='300' align='left'>

## 实例：学习异或 XOR

XOR函数是两个二进制 $x_1,x_2$ 的运算。运算法则是：当这两个二进制值中恰好有一个为 1 时，XOR函数返回值为 1；其余情况返回值为 0 。

下面我们以“学习 XOR函数”来搭建一个可以完整工作的前馈网络。

具体来说，我们引入一个非常简单的前馈神经网络：

* **包含两个单元的隐藏层**：
$$\mathbf{h} = f^{(1)}(\mathbf{x};\mathbf{W},c)=g\circ f  = \max\{0, \mathbf{W}^T\mathbf{x}+c\}$$
其中 $\mathbf{x},\mathbf{W},\mathbf{c}$ 分别对应着输入、权重矩阵、隐藏层的阈值，$\mathbf{h} = (h_1, h_2)$ 为隐藏层的输出；激活函数 g 选取了默认推荐的 [线性整流函数(Rectified Linear Unit, ReLU)](https://zh.wikipedia.org/wiki/%E7%BA%BF%E6%80%A7%E6%95%B4%E6%B5%81%E5%87%BD%E6%95%B0#%E5%B8%A6%E6%B3%84%E9%9C%B2%E7%BA%BF%E6%80%A7%E6%95%B4%E6%B5%81) 

$$g(z) = \max\{0,z\}$$

* **输出层**：
$$y = f^{(2)}(\mathbf{h};\mathbf{w},b) = \mathbf{w}^T\mathbf{h}+b$$

所以，我们整个网络的复合映射如下：

$$f(\mathbf{x};\mathbf{W},\mathbf{c},\mathbf{w},b) = \mathbf{w}^T\max\{0,\mathbf{W}^T\mathbf{x}+c\} + b$$

下图为对应的拓扑结构计算图。

<img src='figure/XOR.png' width='400' height='300' align='left'>

我们可以给出 XOR 函数的一个解，令

$$\begin{split}
\mathbf{W} &= 
\left[                 
\begin{array}{c}   
1 & 1\\  
1 & 1
\end{array} 
\right]\\
\mathbf{c} &= 
\left[                 
\begin{array}{c}   
0\\  
-1
\end{array} 
\right]\\
\mathbf{w} &= 
\left[                 
\begin{array}{c}   
1\\  
-2
\end{array} 
\right]
\end{split}$$

以及 $b=0$

输入矩阵 $\mathbf{X}$ 包含二进制输入空间的四个点，每个样本一行，表示如下：

$$\mathbf{X} = 
\left[                 
\begin{array}{c}   
0 & 0\\  
0 & 1\\
1 & 0\\
1 & 1
\end{array} 
\right]$$

带入隐藏层的映射得

$$\begin{split}
\mathbf{W}^T\mathbf{x}+c &= 
\left[                 
\begin{array}{c}   
1 & 1\\  
1 & 1
\end{array} 
\right]
\left[                 
\begin{array}{c}   
0 & 0 & 1 & 1\\  
0 & 1 & 0 & 1
\end{array} 
\right] + 
\left[                 
\begin{array}{c}   
0\\  
-1
\end{array} 
\right]\\
\\
&= \left[                 
\begin{array}{c}   
0 & 1 & 1 & 2\\  
0 & 1 & 1 & 2
\end{array} 
\right]+ 
\left[                 
\begin{array}{c}   
0\\  
-1
\end{array} 
\right]\\
\\
&=\left[                 
\begin{array}{c}   
0 & 1 & 1 & 2\\  
-1 & 0 & 0 & 1
\end{array} 
\right]
\end{split}$$

经过激活函数 $max\{0,z\}$ 之后，隐藏层的输出为

$$\mathbf{h}=\left[                 
\begin{array}{c}   
0 & 1 & 1 & 2\\  
0 & 0 & 0 & 1
\end{array} 
\right]$$

则输出层为

$$\begin{split}
\mathbf{w}^T\mathbf{h}+c &= 
\left[                 
\begin{array}{c}   
0 &-2
\end{array}
\right]
\left[                 
\begin{array}{c}   
0 & 1 & 1 & 2\\  
0 & 0 & 0 & 1
\end{array} 
\right] + 0\\
\\
&=\left[                 
\begin{array}{c}   
0 &1 & 1 & 0
\end{array}
\right]
\end{split}$$


输出恰好是异或函数的解。

这个例子中，我们简单地指定了解决方案，然后说明它得到的误差为零。实际问题可能会有数十亿的模型参数以及训练样本，不能这样简单猜测。需要使用梯度下降法来求解。

## 基于梯度的学习

通过之前的例子我们可以看到，神经网络和一般的线性模型的最大区别在于，

    神经网络中涉及到的非线性(比如激活函数)会导致常用的代价函数非凸。这意味着无法保证神经网络在任意的初始值下最终都全局收敛，而仅仅只能使得代价函数达到一个比较小的值。换言之，用于非凸损失函数的随机梯度下降没有这种收敛性保证，并且对参数初始值很敏感。
    
    用于训练前馈神经网络以及几乎所有深度模型的迭代求解的梯度类优化算法会在之后详细介绍，包括如何合适地进行参数初始化。

## 代价函数

深度神经网络中一个很重要的环节是代价函数的选择，但实际上神经网络中涉及到的代价函数和机器学习中的代价函数是几乎相同的。

大多数时候，神经网络对应的参数模型定义了一个分布 $p(\mathbf{y}|\mathbf{x};\mathbf{\theta})$，我们可以使用最大似然原理来构建代价函数，由最大似然与最小化交叉熵的等价性，我们可以使用训练数据和模型预测间的交叉熵来作为代价函数。

**用于训练神经网络的完整的代价函数，通常会在我们描述的基础代价函数上加一个正则项，用于线性模型的权重衰减方法也适用于深度神经网络，而且是最常用的正则化策略之一。**

### 使用最大似然学习条件分布

使用最大似然来给出代价函数的优势在于：

1. 减轻了为每个模型设计代价函数的负担，明确一个模型 $p(\mathbf{y}|\mathbf{x};\mathbf{\theta})$ 自动确定了代价函数 $\log p(\mathbf{y}|\mathbf{x};\mathbf{\theta})$;


2. **很多输出单元都会包含一个指数函数，在变量取绝对值非常大的值时会造成饱和(函数会变得非常平)，这会导致代价函数的梯度变得非常小，负对数似然代价函数中的对数函数抑制了某些输出单元中输出的指数所导致的在错误的样本处饱和不利于梯度下降的特性。详细会在下面的“输出单元”环节介绍。**

## 输出单元

## BP反向传播算法